# Demo of collaborative target tracking 

In [ ]:
import collaborative_bearing_estimation as cbe
import numpy as np

# %matplotlib notebook
%matplotlib Qt5
from matplotlib import pyplot as plt
from matplotlib.patches import Ellipse
import datetime

## setting up the scenario


In [ ]:

seed = np.random.randint(0,200)
print (seed)


#seed = 112 is not great for BSI vs CI possibly due to no implementation of rejecting inconsistent stuff
# seed= 67 # is a good case 
# seed= 62 # great for BSI but CI performs worse than KF possibly due to no rejection of inconsistent stuff
# seed = 197 # not great for BSI vs CI and almost matching Kalman
seed = 112
np.random.seed (seed)

#initialize figure
x_range_plot = np.array([[-15],[15]])
y_range_plot = np.array([[-20],[20]])









# agent 1 and 2 list of positions with initial position added (same starting position for both)
x1 = np.array([[-10],[10]])
x2 = np.array([[-10],[10]])

# target
x = np.array([[0],[0]])



s1 = 1 # linear velocity
a1 = np.deg2rad(0) #steering angle
xh1 = np.array([[3],[2]])
e1 = 25
P1 = e1 * np.eye(2)
P1_inv = 1/e1 *np.eye(2)


s2 = 1
a2 = np.deg2rad(-90)
xh2 = np.array([[2],[2]])
e2 = 25
P2 = e2 * np.eye(2)
P2_inv = 1/e2 *np.eye(2)


dt = 1
T = 100


#create two agents that estimate the target
a_l_NC = []
a_l_C = []

a1_NC = cbe.Observer(1, x1, s1, a1, 4, 20, np.deg2rad(5), xh1, P1, P1_inv, None)
a1_BSI = cbe.Observer(1, x1, s1, a1, 4, 20, np.deg2rad(5), xh1, P1, P1_inv, "BSI")
a1_CI = cbe.Observer(1, x1, s1, a1, 4, 20, np.deg2rad(5), xh1, P1, P1_inv, "CI")
a1_Kalman = cbe.Observer(1, x1, s1, a1, 4, 20, np.deg2rad(5), xh1, P1, P1_inv, "Kalman")

a_l_NC.append(a1_NC)
a_l_C.append(a1_BSI)
a_l_C.append(a1_CI)
a_l_C.append(a1_Kalman)

a2_NC = cbe.Observer(2, x2, s2, a2, 4, 20, np.deg2rad(5), xh2, P2, P2_inv, None)
a2_BSI = cbe.Observer(2, x2, s2, a2, 4, 20, np.deg2rad(5), xh2, P2, P2_inv, "BSI")
a2_CI = cbe.Observer(2, x2, s2, a2, 4, 20, np.deg2rad(5), xh2, P2, P2_inv, "CI")
a2_Kalman = cbe.Observer(2, x2, s2, a2, 4, 20, np.deg2rad(5), xh2, P2, P2_inv, "Kalman")

a_l_NC.append(a2_NC)
a_l_C.append(a2_BSI)
a_l_C.append(a2_CI)
a_l_C.append(a2_Kalman)





## simulation run

In [ ]:



fig = plt.figure()
ax = fig.add_subplot(111, aspect='equal')
ax.set_xlim(x_range_plot[0,0], x_range_plot[1,0])
ax.set_ylim(y_range_plot[0,0], y_range_plot[1,0])


e_1 = np.random.normal(0, a_l_NC[0].th_e, size= int(T/dt)) # agent 1's entire measurement error
e_2 = np.random.normal(0, a_l_NC[1].th_e, size= int(T/dt)) 

for a in a_l_C + a_l_NC :
    a.est_err(x) # capture the first estimation error
    
ax.scatter(x[0,0], x[1,0], s = 100, marker="^", alpha=.5, c = "g", label= "Target")  
for a in a_l_NC:
    a.plot_pos(ax, True)
for t in range (int(T/dt)):
    
    print ("simulation time = ", t)
    for a in a_l_NC:
            a.plot_pos(ax)
    
            
    for a in a_l_NC + a_l_C:
        
                
    
        # plot estimate of agent
#         a.plot_est(ax)
        if a.ID ==1:
        
            cm, Pm, Pm_inv = a.sense(x, e_1[t])
        else: 
            cm, Pm, Pm_inv = a.sense(x, e_2[t])
        
        # plot meas set
#         a.plot_meas(cm, Pm, ax)
        
        a.estimate(cm, Pm, Pm_inv)
        a.est_err(x)
    
    for a in a_l_C + a_l_NC : # after everyone has sensed then we can allow information exchange and move before the next round
        
        for m in [k for k in a_l_C if k.ID != a.ID and k.fuse_method == a.fuse_method ]: #only looking in collaborative agents with matching method
            
            print ("fusing agent {0}{1} with agent{2}{3}".format(a.ID,a.fuse_method, m.ID, m.fuse_method))
            
            a.fuse (m)
        
        a.move(dt)
        
        
        
#         plt.pause(.8)
#         plt.show()
        
        
for a in a_l_C + a_l_NC:
    
    a.plot_est(ax)
    

ax.legend()    

In [ ]:
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
plt.title ( "Estimation Error")

fig3 = plt.figure()
ax3 = fig3.add_subplot(111)
plt.title ( "Estimation Confidence")

for a in a_l_C + a_l_NC:
    if a.fuse_method == "Kalman":
        color = 'g'
    if a.fuse_method == "BSI":
        color = 'blue'
    if a.fuse_method == "CI":
        color = "r"
    if a.fuse_method == None:
        color ='k'
        
    if a.ID ==1:
        ls = "-"
    else:
        ls = "--"

    ax2.plot(a.err_l, color=color, ls=ls, label="Agent_{0}, Method_{1}".format(a.ID, a.fuse_method))
    ax3.plot(a.conf_l, color=color, ls=ls, label="Agent_{0}, Method_{1}".format(a.ID, a.fuse_method))
    
ax2.legend()
ax3.legend()